## I. Data Preparation (train)

The goal of this dataset is to clean the data set we'll use for data visualizations and training the model. Note that further data wrangling is required for training the model, which will be done in jupyter notebook no.2 from this project.
    
Here's what to expect on this notebook:
    - Importing libraries and data
    - Fixing data types
    - Find number of nulls
    - Feature engineering: Creating new columns, aggreagating categories and hot encoding
    
Finally we save modified data for future data visualizations


### 1. Importing libraries and data

In [1]:
# import libraries
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math

%matplotlib inline
%matplotlib inline

In [2]:
#import first data set (CPtrain values)
df=pd.read_csv('CPtrain_values.csv',sep=',')
df.shape

(1401, 46)

In [3]:
#View data
df.head(2)

,row_id,country_code,year,agricultural_land_area,percentage_of_arable_land_equipped_for_irrigation,cereal_yield,droughts_floods_extreme_temps,forest_area,total_land_area,fertility_rate,...,open_defecation,hiv_incidence,rail_lines_density,access_to_electricity,co2_emissions,unemployment_rate,total_labor_force,military_expenditure_share_gdp,proportion_of_seats_held_by_women_in_gov,political_stability
0,0,889f053,2002,235077.658128,38.558520,935.754365,NaN,5397.737544,537408.312513,5.928981,...,28.933529,0.009975,NaN,52.433239,15485.148165,14.788020,4.350521e+06,7.021072,0.698153,-1.393801
1,1,9e614ab,2012,23000.640738,21.282631,4031.452161,NaN,178336.529912,232013.631676,3.000629,...,29.665789,0.029840,NaN,77.715574,1696.458184,0.687018,3.262614e+06,0.198831,25.101826,0.030556


In [4]:
# Remove duplicates if available (row)
print(df.shape)
print (df.row_id.unique().shape)

(1401, 46)
(1401,)


No duplicates to be removed (based on id column)

### 2.  Fixing data types and see number of nulls

In [5]:
#view data types and see if there's anything wrong
df.dtypes

row_id                                                 int64
country_code                                          object
year                                                   int64
agricultural_land_area                               float64
percentage_of_arable_land_equipped_for_irrigation    float64
cereal_yield                                         float64
droughts_floods_extreme_temps                        float64
forest_area                                          float64
total_land_area                                      float64
fertility_rate                                       float64
life_expectancy                                      float64
rural_population                                     float64
total_population                                     float64
urban_population                                     float64
population_growth                                    float64
avg_value_of_food_production                         float64
cereal_import_dependency

All columns seem to be the right data type

In [6]:
#see how many nulls we have on data
df.isna().sum()

row_id                                                  0
country_code                                            0
year                                                    0
agricultural_land_area                                 16
percentage_of_arable_land_equipped_for_irrigation     248
cereal_yield                                           64
droughts_floods_extreme_temps                        1326
forest_area                                            16
total_land_area                                         0
fertility_rate                                         14
life_expectancy                                        15
rural_population                                        0
total_population                                        0
urban_population                                        0
population_growth                                       1
avg_value_of_food_production                          167
cereal_import_dependency_ratio                        317
food_imports_a

Considering I have 1401 rows, having columns with more than 600 nulls is a concern. I will take a different approach to see how many the columns with fewer rows filled in.

In [7]:
# See number of rows with non nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1401 entries, 0 to 1400
Data columns (total 46 columns):
row_id                                               1401 non-null int64
country_code                                         1401 non-null object
year                                                 1401 non-null int64
agricultural_land_area                               1385 non-null float64
percentage_of_arable_land_equipped_for_irrigation    1153 non-null float64
cereal_yield                                         1337 non-null float64
droughts_floods_extreme_temps                        75 non-null float64
forest_area                                          1385 non-null float64
total_land_area                                      1401 non-null float64
fertility_rate                                       1387 non-null float64
life_expectancy                                      1386 non-null float64
rural_population                                     1401 non-null float64


The following rows have very few values:
- Adult literacy rate (2085)
- droughts_floods_extreme_temps (75)
- inequality_index (429)
- rail_lines_density (457)
- school_enrollment_rate_female (795)                        
- school_enrollment_rate_total (897)                       
- tax_revenue_share_gdp (856)

I need to do something about them. If I do imputation (replacing nulls with a value, such as the mode, median, mean, or any other one) would bias the prediction results. On the other hand, if I remove the rows with missing values, I would end up with a dataset with only 75 values.

Probably, the best thing to do is to remove those columns. Before that, I shall make sure that the correlation between these columns (features) I want to remvoe and the label (prevalence of undernourishment) is not relevant.


In [8]:
# Find the most important correlations

import pandas as pd
d = df.loc[:, df.columns != 'country_code']

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(d, 20))

Top Absolute Correlations
total_population               total_labor_force                0.997911
rural_population               total_population                 0.993388
                               total_labor_force                0.989707
school_enrollment_rate_female  school_enrollment_rate_total     0.979683
urban_population               total_labor_force                0.974371
total_population               urban_population                 0.972803
agricultural_land_area         total_land_area                  0.954624
rural_population               urban_population                 0.940436
forest_area                    total_land_area                  0.928500
access_to_improved_sanitation  access_to_electricity            0.863503
urban_population               co2_emissions                    0.850898
adult_literacy_rate            school_enrollment_rate_female    0.829266
life_expectancy                access_to_electricity            0.827314
fertility_rate           

As expected, amongst the top 20 most important correlations with the label I cannot find any of the feautres with very few non-null values. 

Therefore, I proceed to remove them.

In [9]:
#Drop Columns with too many missing values (nan)
df= df.drop(['adult_literacy_rate','droughts_floods_extreme_temps','hiv_incidence','inequality_index','rail_lines_density',
             'school_enrollment_rate_female','school_enrollment_rate_total','tax_revenue_share_gdp'], axis = 1)

In [10]:
df.shape

(1401, 38)

Now the number of columns has decreased from 46 to 38

### 3. Feature Engineering

I will create one 3 new columns:


#### 3.1 Create a categorical column (yearnew)

The reason I create this column is to show that PoU has increased in the 2010 compared to 2000.

In [11]:
#Aggregate Categories for year

def year_xform(al):
    if al > 2009: return '2010'
    else: return '2000'

# Create new column 

df["year2"] = df['year'].map(year_xform)


df['year2'].value_counts()

2000    890
2010    511
Name: year2, dtype: int64

In [12]:
# Now I can remove the year (original) column 
df= df.drop(['year'], axis = 1)

####  3. 2 Create new categorical columns 

In [13]:
## Create new column for urban share
df['urban_share']=df['urban_population']/df['total_population']
df['urban_share'].unique()

array([0.26913152, 0.35160797, 0.3195086 , ..., 0.55484036, 0.24540367,
       0.25542912])

In [14]:
# Create a New column for rural_share...see if it works like urban_share
df['rural_share']=df['rural_population']/df['total_population']

In [15]:
# I should have a total of 40 columns
df.shape

(1401, 40)

### 4. Save modified dataset

In [16]:
#export data set to main directory to be used in a different jupyter notebook
df.to_csv("dfprepared.csv", index=False)